# Environment Setup

In [1]:
!pip install google-cloud-aiplatform[agent_engines,adk]>=1.112.0
!pip install --upgrade google-genai

import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
GOOGLE_CLOUD_PROJECT_ID="data-vpc-sc-demo" #@param {type:"string"}
GOOGLE_CLOUD_PROJECT_NUMBER="1083677030545" #@param {type:"string"}
GOOGLE_CLOUD_LOCATION="us-central1" #@param {type:"string"}
GOOGLE_CLOUD_ZONE="us-central1-c" #@param {type:"string"}
GOOGLE_CLOUD_STORAGE_BUCKET="gs://data-vpc-sc-demo" #@param {type:"string"}

VERTEX_SERVICE_ACCOUNT = "serviceAccount:service-" + str(GOOGLE_CLOUD_PROJECT_NUMBER) + "@gcp-sa-aiplatform.iam.gserviceaccount.com"


In [2]:
from google.colab import auth

auth.authenticate_user(project_id={GOOGLE_CLOUD_PROJECT_ID})

# Application Default Credentials are necessary to get an identity token to call
# Cloud Run.
!gcloud auth application-default login


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  

Command killed by keyboard interrupt

^C


In [3]:
!gcloud services enable "compute.googleapis.com"
!gcloud services enable "aiplatform.googleapis.com"


!gcloud projects add-iam-policy-binding {GOOGLE_CLOUD_PROJECT_ID} \
    --member={VERTEX_SERVICE_ACCOUNT} \
    --role=roles/compute.networkAdmin
!gcloud projects add-iam-policy-binding {GOOGLE_CLOUD_PROJECT_ID} \
    --member={VERTEX_SERVICE_ACCOUNT} \
    --role=roles/dns.peer

 [1] EXPRESSION=resource.name.endsWith('locations/global/buckets/_Default'), TITLE=serviceAccount:service-org-579419163830@gcp-sa-logging.iam.gserviceaccount.com
 [2] None
 [3] Specify a new condition
The policy contains bindings with conditions, so specifying a condition is 
required when adding a binding. Please specify a condition.:  1

Updated IAM policy for project [data-vpc-sc-demo].
auditConfigs:
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_WRITE
  - logType: DATA_READ
  service: cloudscheduler.googleapis.com
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_WRITE
  - logType: DATA_READ
  service: cloudtasks.googleapis.com
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_READ
  - logType: DATA_WRITE
  service: aiplatform.googleapis.com
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_READ
  - logType: DATA_WRITE
  service: notebooks.googleapis.com
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_READ
  - logType: DAT

# Private Network Setup

In [4]:
PVT_NETWORK = 'agentspace-home-network' #@param {type:"string"}
SUBNETWORK = 'agentspace-home-subnet' #@param {type:"string"}
PSC_ATTACHMENT = 'agentspace-psc-attach' #@param {type:"string"}
DNS_PEERING_ZONE = "agentspace-dns-zone"
DNS_DOMAIN_NAME = "privatedb.app."
DNS_DOMAIN_ALL_RECORD = "*." + DNS_DOMAIN_NAME



In [5]:
#Create Network and Subnets
!gcloud compute networks create {PVT_NETWORK} --subnet-mode=custom
!gcloud compute networks subnets create {SUBNETWORK} --network={PVT_NETWORK} --region={GOOGLE_CLOUD_LOCATION} --range="10.0.0.0/24"

#Create Private Service Connect attachment in the subnet - to be used by AgentEngine later
!gcloud compute network-attachments create {PSC_ATTACHMENT} --region={GOOGLE_CLOUD_LOCATION} --connection-preference=ACCEPT_MANUAL --subnets={SUBNETWORK}

#SSH, RDP, ICMP from Everywhere
!gcloud compute firewall-rules create {PVT_NETWORK}-ssh --network {PVT_NETWORK} --allow tcp:22,tcp:3389,icmp

# Alllow SSH with IAP proxy
!gcloud compute firewall-rules create {PVT_NETWORK}-allow-ssh-from-iap  --direction=INGRESS --action=allow --rules=tcp:22 --network={PVT_NETWORK} --source-ranges=35.235.240.0/20

#Allow TCP, UDP-All ports only from within the network
!gcloud compute firewall-rules create {PVT_NETWORK}-tcp --direction=INGRESS --network {PVT_NETWORK} --allow tcp:0-65535,udp:0-65535,icmp --source-ranges 10.0.0.0/24 --priority 65534



ERROR: (gcloud.compute.networks.create) Could not fetch resource:
 - The resource 'projects/data-vpc-sc-demo/global/networks/agentspace-home-network' already exists

ERROR: (gcloud.compute.networks.subnets.create) Could not fetch resource:
 - The resource 'projects/data-vpc-sc-demo/regions/us-central1/subnetworks/agentspace-home-subnet' already exists

ERROR: (gcloud.compute.network-attachments.create) Could not fetch resource:
 - The resource 'projects/data-vpc-sc-demo/regions/us-central1/networkAttachments/agentspace-psc-attach' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/data-vpc-sc-demo/global/firewalls/agentspace-home-network-ssh' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/data-vpc-sc-demo/global/firewalls/agentspace-home-network-allow-ssh-from-iap' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The

In [6]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT_ID}

Updated property [core/project].


In [7]:
VM_NAME = "pvt-db-instance2"

!gcloud compute instances create {VM_NAME} \
    --project={GOOGLE_CLOUD_PROJECT_ID} \
    --zone={GOOGLE_CLOUD_ZONE} \
    --machine-type=e2-medium \
    --no-address \
    --shielded-secure-boot \
    --tags=http-server,https-server,lb-health-check \
    --image-family=debian-12 \
    --image-project=debian-cloud \
    --boot-disk-size=10GB \
    --boot-disk-type=pd-standard \
    --network=agentspace-home-network \
    --subnet=agentspace-home-subnet \
    --metadata=enable-oslogin=true,startup-script='sudo apt update sudo apt -y install postgresql postgresql-client postgresql-contrib sudo -u postgres psql postgres' \
    --scopes=cloud-platform \
    --shielded-secure-boot \
    --shielded-vtpm \
    --shielded-integrity-monitoring \
    --provisioning-model=STANDARD \
    --no-address



ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/data-vpc-sc-demo/zones/us-central1-c/instances/pvt-db-instance2' already exists



In [8]:
# VM's private Internal IP
VM_PRIVATE_IP = "10.0.0.5" #@param {type:"string"}

In [9]:
!gcloud dns --project={GOOGLE_CLOUD_PROJECT_ID} managed-zones create {DNS_PEERING_ZONE} --description="AS-PVT-CONNECTIVITY" --dns-name={DNS_DOMAIN_NAME} --visibility="private" --networks={PVT_NETWORK}

!gcloud dns record-sets transaction start --zone={DNS_PEERING_ZONE}
!gcloud dns record-sets transaction add {VM_PRIVATE_IP} --name={DNS_DOMAIN_ALL_RECORD} --ttl="300" --type="A" --zone={DNS_PEERING_ZONE}
!gcloud dns record-sets transaction add {VM_PRIVATE_IP} --name={DNS_DOMAIN_NAME} --ttl="300" --type="A" --zone={DNS_PEERING_ZONE}
!gcloud dns record-sets transaction execute --zone={DNS_PEERING_ZONE}

ERROR: (gcloud.dns.managed-zones.create) Resource in projects [data-vpc-sc-demo] is the subject of a conflict: The resource 'entity.managedZone' named 'agentspace-dns-zone' already exists
Transaction started [transaction.yaml].
Record addition appended to transaction at [transaction.yaml].
Record addition appended to transaction at [transaction.yaml].
ERROR: (gcloud.dns.record-sets.transaction.execute) HTTPError 409: The resource 'entity.change.additions[privatedb.app.][A]' named 'privatedb.app. (A)' already exists


# Setup Database on the VM created

1. Before the database setup, create a Cloud NAT gateway for this network. This will be disabled later.

2. Connect to the VM created above through SSH

3. Run the following instructions inside the terminal

```
sudo apt update

sudo apt -y install postgresql postgresql-client postgresql-contrib

sudo -u postgres psql postgres

```

4. Inside the Postgres prompt, execute these

```
\password postgres

CREATE EXTENSION adminpack;

CREATE TABLE cars (
  brand VARCHAR(255) NOT NULL,
  model VARCHAR(255) NOT NULL,
  year INT NOT NULL
);

INSERT INTO cars (brand, model, year)
VALUES
  ('Volvo', 'p1800', 1968),
  ('BMW', 'M1', 1978),
  ('Toyota', 'Celica', 1975);

SELECT * FROM cars;

```
5. Enter \q to exit PSQL
6. Edit /etc/postgresql/15/main/pg_hba.conf to add following lines at the end. To find out IP address for your setup, go to Console > Private Service Connect > Network Attachment > Connected Projects > Endpoints. In case you do not see Connected Projects yet, wait till you deploy Agent-Engine and then repeat this step

```
host   all    all     10.0.0.3/32      md5
host   all    all     10.0.0.4/32      md5
```
7. Edit /etc/postgresql/15/main/postgresql.conf

```
Search for
#listen_addresses = 'localhost'
Replace with
listen_addresses = '*'

```
8. sudo service postgresql restart

9. Delete the Cloud NAT gateway

In [10]:
# Modify Network firewall to allow Postgres Traffic from Private Service Connect
# To find out source range address for your setup, go to Console > Private Service Connect > Network Attachment > Connected Projects > Endpoints.
# In case you do not see Connected Projects yet, wait till you deploy Agent-Engine and then repeat this step

!gcloud compute firewall-rules create {PVT_NETWORK}-postgres --network {PVT_NETWORK} --allow tcp:5432 --source-ranges=10.0.0.4/32,10.0.0.5/32 --priority 1000

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/data-vpc-sc-demo/global/firewalls/agentspace-home-network-postgres' already exists



# Setup the agent

In [11]:
import vertexai
import os
from google.adk.agents import LlmAgent
from vertexai.preview.reasoning_engines import AdkApp
import json
import psycopg2, psycopg2.extras

google_cloud_project = GOOGLE_CLOUD_PROJECT_ID
google_cloud_location = GOOGLE_CLOUD_LOCATION
staging_bucket = GOOGLE_CLOUD_STORAGE_BUCKET
pvt_network = PVT_NETWORK
domain_name= DNS_DOMAIN_NAME
psc_attachment= PSC_ATTACHMENT
db_instance = DNS_DOMAIN_NAME

def get_options_from_database():
    """
    Connect to PostgreSQL database, query the 'cars' table, and return results as JSON.
    Returns:
        str: JSON string containing the query results
    """
    # Database connection parameters
    db_config = {
        'host': db_instance,
        'database': 'postgres',
        'user': 'postgres',
        'password': 'postgres',
        'port': '5432'
    }

    connection = None
    try:
        # Establish connection
        connection = psycopg2.connect(**db_config)
        # Create cursor with dictionary-like access
        cursor = connection.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
        # Execute query
        cursor.execute("SELECT * FROM public.cars2")
        # Fetch all results
        results = cursor.fetchall()
        # Convert to JSON
        json_results = json.dumps(results, default=str, indent=2)
        print(json_results)
        return json_results

    except psycopg2.Error as e:
        print(f"Database error: {e}")
        return json.dumps({"error": str(e)})
    except Exception as e:
        print(f"Error: {e}")
        return json.dumps({"error": str(e)})
    finally:
        # Clean up
        if connection:
            cursor.close()
            connection.close()

root_agent = LlmAgent(
    name="root_agent",
    model="gemini-2.5-pro",
    description="A helpful automotive sales agent",
    instruction="""
        You are a helpful automotive vehicle supplier and sales agent. Greet the user respectfully.

        Whenever user talks about cars, use the tool get_cars_from_db to tell them the available options

    """,
    tools=[get_options_from_database]
)

In [12]:
#Deploy the Agent on Agent-Engine

def deploy_agent_engine_app(mode: str):
  app = AdkApp(
      agent=root_agent,
      enable_tracing=True,
  )

  client = vertexai.Client(
      project=google_cloud_project,
      location=google_cloud_location,
  )
  print(f"Vertext AI client initiated {client}")

  agent_config = {
      "staging_bucket": staging_bucket,
      "requirements": [
          "google-cloud-aiplatform[agent_engines,adk]",
          "cloudpickle==3.1.1",
          "python-dotenv",
          "deprecated",
          "absl-py",
          "pydantic",
          "psycopg2-binary",
        ],
        "psc_interface_config": {
            "network_attachment": psc_attachment,
            "dns_peering_configs": [
                {
                    "domain": domain_name,
                    "target_project": google_cloud_project,
                    "target_network": pvt_network,
                },
            ],
        },
  }

  if mode =="CREATE":
    print("Creating Agent")
    remote_agent = client.agent_engines.create(
       agent = app,
       config= agent_config,
    )
  else:
    print("Updating Agent")
    remote_agent = client.agent_engines.update(
       agent = app,
       name = "<agent-engine-resource-name>",
       config= agent_config,
    )
  return remote_agent

In [ ]:
# Choose CREATE or UPDATE depending on your use case
remote_agent = deploy_agent_engine_app("CREATE")

Vertext AI client initiated <vertexai._genai.client.Client object at 0x7c291d576a10>
Creating Agent


INFO:vertexai_genai.agentengines:Identified the following requirements: {'cloudpickle': '3.1.1', 'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.118.0'}
INFO:vertexai_genai.agentengines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]', 'cloudpickle==3.1.1', 'python-dotenv', 'deprecated', 'absl-py', 'pydantic', 'psycopg2-binary']
INFO:vertexai_genai.agentengines:Using bucket data-vpc-sc-demo
INFO:vertexai_genai.agentengines:Wrote to gs://data-vpc-sc-demo/agent_engine/agent_engine.pkl
INFO:vertexai_genai.agentengines:Writing to gs://data-vpc-sc-demo/agent_engine/requirements.txt
INFO:vertexai_genai.agentengines:Creating in-memory tarfile of extra_packages
INFO:vertexai_genai.agentengines:Writing to gs://data-vpc-sc-demo/agent_engine/dependencies.tar.gz
INFO:vertexai_genai.agentengines:View progress and logs at https://console.cloud.google.com/logs/query?project=data-vpc-sc-demo.


In [ ]:
import requests
import json

api = remote_agent.api_resource.name
AGENT_RESOURCE_ID = api.split("/")[-1]
print(AGENT_RESOURCE_ID)
token = !gcloud auth application-default print-access-token
print(token[0])
# https://us-central1-aiplatform.googleapis.com/v1/projects/data-vpc-sc-demo/locations/us-central1/reasoningEngines/6339542153198305280:query
print(api)
response = requests.post(
    # f"https://us-central1-aiplatform.googleapis.com/v1/{api}:query",
    "https://us-central1-aiplatform.googleapis.com/v1/projects/data-vpc-sc-demo/locations/us-central1/reasoningEngines/6339542153198305280:query",
    headers={
        "Content-Type": "application/json; charset=utf-8",
        "Authorization": f"Bearer {token[0]}"
    },
    data=json.dumps({ "class_method": "async_create_session", "input": {"user_id": "user_12"} })
)
print(response.text)



In [ ]:
# Query the deployed Agent

response = requests.post(
    f"https://us-central1-aiplatform.googleapis.com/v1beta1/projects/{GOOGLE_CLOUD_PROJECT_ID}/locations/{GOOGLE_CLOUD_LOCATION}/reasoningEngines/{AGENT_RESOURCE_ID}:streamQuery?alt=sse -d",
    headers={
        "Content-Type": "application/json; charset=utf-8",
        "Authorization": f"Bearer {token[0]}"
    },
    data=json.dumps({ "class_method": "stream_query", "input": {"user_id": "user_12","session_id": "{SESSION_ID}","message": "Tell me about the model of available cars"} })
)
print(response.content)

b''


In [ ]:
# Create a new session for the deployed agent.
# Session id from this response is needed in the next step to query the remote agent

# AGENT_RESOURCE_ID = "6339542153198305280" #@param {type:"string"}
import requests
import json

token = !gcloud auth application-default print-access-token

response = requests.post(
    f"https://us-central1-aiplatform.googleapis.com/v1beta1/projects/{GOOGLE_CLOUD_PROJECT_ID}/locations/{GOOGLE_CLOUD_LOCATION}/reasoningEngines/{AGENT_RESOURCE_ID}:query",
    headers={
        "Content-Type": "application/json; charset=utf-8",
        "Authorization": f"Bearer {token[0]}"
    },
    data=json.dumps({ "class_method": "async_create_session", "input": {"user_id": "user_12"} })
)
print(response.text)

{
  "error": {
    "code": 500,
    "message": "Unknown Error.",
    "status": "UNKNOWN"
  }
}



In [ ]:
# Query the deployed Agent

response = requests.post(
    f"https://us-central1-aiplatform.googleapis.com/v1beta1/projects/{GOOGLE_CLOUD_PROJECT_ID}/locations/{GOOGLE_CLOUD_LOCATION}/reasoningEngines/{AGENT_RESOURCE_ID}:streamQuery?alt=sse -d",
    headers={
        "Content-Type": "application/json; charset=utf-8",
        "Authorization": f"Bearer {token[0]}"
    },
    data=json.dumps({ "class_method": "stream_query", "input": {"user_id": "user_12","session_id": "{SESSION_ID}","message": "Tell me about the model of available cars"} })
)
print(response.text)

# Register the agent with AgentSpace

In [ ]:
%%bash
export GOOGLE_CLOUD_PROJECT_NUMBER="1083677030545" #@param {type:"string"}
export REASONING_ENGINE="5214768146262523904" #@param {type:"string"}
export DISPLAY_NAME="Database Agent"
export DESCRIPTION="Queries Private DB Instance"
export TOOL_DESCRIPTION="Queries Private DB Instance"
export AS_APP="Test" #@param {type:"string"}

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "X-Goog-User-Project: ${GOOGLE_CLOUD_PROJECT_NUMBER}" \
https://us-discoveryengine.googleapis.com/v1beta/projects/${GOOGLE_CLOUD_PROJECT_NUMBER}/locations/us/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents \
  -d '{
      "displayName": "'"${DISPLAY_NAME}"'",
      "description": "'"${DESCRIPTION}"'",
      "adk_agent_definition": {
        "tool_settings": {
          "tool_description": "'"${TOOL_DESCRIPTION}"'"
        },
        "provisioned_reasoning_engine": {
          "reasoning_engine":
            "'"${REASONING_ENGINE}"'"
        }
      }
  }'


{
  "error": {
    "code": 404,
    "message": "Method not found.",
    "status": "NOT_FOUND"
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450    0   100  100   350   1143   4003 --:--:-- --:--:-- --:--:--  5172
